<a href="https://colab.research.google.com/github/tripidhoble/Hackathon-Projects/blob/master/MFC_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [0]:
#import important libraries
from google.colab import files
import copy

In [0]:
path = "drive/My Drive/data/"

customer_df = pd.read_csv(path + 'customer.csv')
invoice_df = pd.read_csv(path + 'invoice.csv')
jtd_df = pd.read_csv(path + 'jtd.csv')
plant_df = pd.read_csv(path + 'plant.csv')
vehicles_df = pd.read_csv(path + 'vehicles.csv')

In [0]:
# 1. drop unwanted columns from all dataframes
# 2. lowercase all the categorical columns

def drop_col(dataset, col_to_drop):
    dataset.drop(col_to_drop, axis=1, inplace=True)
    return dataset

def lowercase(dataset):
    categorical_col = dataset.select_dtypes(exclude=['number']).columns
    for col in categorical_col:
        dataset[col] = dataset[col].map(lambda x: x if type(x)!=str else x.lower())
    return dataset

invoice_columns_to_drop = ['Unnamed: 0', 'Amt Rcvd From Custom', 'Amt Rcvd From Ins Co','Area / Locality',
                           'CGST(14%)', 'CGST(2.5%)', 'CGST(6%)', 'CGST(9%)',
                           'IGST(12%)', 'IGST(18%)', 'IGST(28%)', 'IGST(5%)', 'Insurance Company',
                           'Outstanding Amt', 'SGST/UGST(14%)', 'SGST/UGST(2.5%)', 'SGST/UGST(6%)', 'SGST/UGST(9%)',
                           'Service Advisor Name', 'TDS amount', 'Total CGST', 'Total GST', 'Total IGST',
                           'Total SGST/UGST'
                          ]
customer_columns_to_drop = ['Unnamed: 0','Death date']
plant_columns_to_drop = ['Unnamed: 0','Name 1','Factory calendar','Valuation Area', 'Customer no. - plant','PO Box','Postal Code','Name 2','Vendor number plant','House number and street']
jtd_columns_to_drop = ['Unnamed: 0']
vehicles_columns_to_drop = ['Unnamed: 0','Product GUID']

dataframes = [invoice_df, customer_df, plant_df, jtd_df, vehicles_df]
dataframe_columns_to_drop = [invoice_columns_to_drop, customer_columns_to_drop, plant_columns_to_drop, jtd_columns_to_drop, vehicles_columns_to_drop]

for i in range(len(dataframes)):
  dataframes[i] = drop_col(dataframes[i],dataframe_columns_to_drop[i])
  dataframes[i] = lowercase(dataframes[i])

In [0]:
# Preprocessing for dataset merge operation

# 1. Remove leading zeros from 'Customer No.' column
def remove_leading_zeros(dataset,cols):
    for col in cols:
      dataset[col] = dataset[col].astype(str).apply(lambda x: x.lstrip("0"))
    return dataset
invoice_df  = remove_leading_zeros(invoice_df,['Customer No.'])
customer_df = remove_leading_zeros(customer_df,['Customer No.'])


# 2. Rename columns to use them as key column for merge operation
def rename_columns(dataset, rename_cols_dict):
    dataset.rename(columns=rename_cols_dict, inplace=True)
    return dataset
dict_invoice_df = {'District':'State'}
dict_vehicles_df = {'Vehicle Model':'Make', 'License Plate Number':'Regn No'}

invoice_df  = rename_columns(invoice_df, dict_invoice_df)
vehicles_df = rename_columns(vehicles_df, dict_vehicles_df)


# 3. Replace values in key columns for merge operation
def replacement(dataset, col, dict_replacement, regex=False):
    dataset[col] = dataset[col].replace(dict_replacement, regex=regex )
    return dataset
make_replacements = {
                    'mahindra &  mahindra': 'mahindra',
                    'tata motors': 'tata',
                    'maruti suzuki': 'maruti',
                    'mercedes benz': 'mercedes-benz',
                    'porche': 'porsche',
                    'land rover' : 'rover',
                    'mitsubishi motors':'mitsubishi motor',
                    'premier\xa0automobiles': 'premierauto'
                    }
pattern_replacement = {'z_':''}
vehicles_df = replacement(vehicles_df, 'Make', pattern_replacement, regex=True)
vehicles_df = replacement(vehicles_df, 'Make', make_replacements)
invoice_df  = replacement(invoice_df, 'Make', make_replacements)

In [0]:
# Merge dataframes based on key
def merge_df(left_df, right_df, key):
    merge_df = pd.merge(left=left_df, right=right_df, how='left', on=key)
    return merge_df

invoice_customer_df = merge_df(left_df=invoice_df, right_df=customer_df,key=['Customer No.'])
invoice_customer_plant_df = merge_df(left_df=invoice_customer_df, right_df=plant_df,key=['Plant','State'])
invoice_customer_plant_vehicles_df = merge_df(left_df=invoice_customer_plant_df, right_df=vehicles_df,key=['Regn No','Make'])

master_df = invoice_customer_plant_vehicles_df

In [25]:
print("invoice_df shape: ",invoice_df.shape)
print("customer_df shape: ",customer_df.shape)
print("invoice_customer_df shape: ",invoice_customer_df.shape)
print("plant_df shape: ",plant_df.shape)
print("invoice_customer_plant_df shape: ",invoice_customer_plant_df.shape)
print("vehicles_df shape: ",vehicles_df.shape)
print("invoice_customer_plant_vehicles_df shape: ",invoice_customer_plant_vehicles_df.shape)

invoice_df shape:  (984741, 35)
customer_df shape:  (555338, 8)
invoice_customer_df shape:  (984741, 42)
plant_df shape:  (438, 4)
invoice_customer_plant_df shape:  (984741, 44)
vehicles_df shape:  (588761, 3)
invoice_customer_plant_vehicles_df shape:  (992903, 45)


In [0]:
# fill missing values of 'CITY' column 
master_df['CITY'].fillna(master_df['City'], inplace = True)

#remove dulicate column 'City'
master_df = drop_col(dataset=master_df, col_to_drop=['City'])

In [26]:
# Function to identify the number of missing values in every feature
def missing_datas(dataset):
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = ((dataset.isnull().sum())*100/dataset.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data
    
missing_data_master_df = missing_datas(master_df)
print("missing data for 'master_df': ")
print(missing_data_master_df[missing_data_master_df['Percent']>0.00])

missing_data_jtd_df = missing_datas(jtd_df)
print("missing data for 'jtd_df': ")
print(missing_data_jtd_df[missing_data_jtd_df['Percent']>0.00])

missing data for 'master_df': 
                      Total    Percent
Claim No.            988053  99.511533
Occupation           965665  97.256731
Cash /Cashless Type  965197  97.209597
Date of Birth        962418  96.929710
Marital Status       958174  96.502277
Gate Pass Date       946518  95.328345
Policy no.           924141  93.074651
Expiry Date          894807  90.120284
Technician Name      734250  73.949822
Total Value          690834  69.577189
ODN No.              690834  69.577189
Sales organization   279168  28.116342
City                 271772  27.371455
Title                152663  15.375419
Fuel Type             21738   2.189338
Data Origin            7627   0.768152
Partner Type           5489   0.552823
Business Partner       3510   0.353509
Regn No                3315   0.333869
Model                  1736   0.174841
Cust Type                 4   0.000403
CITY                      4   0.000403
missing data for 'jtd_df': 
                       Total    Percent
Labo

In [29]:
# Function to drop missing values
def drop_missing(dataset, missing, min_threshold, max_threshold):
    dataset = dataset.drop((missing[missing['Percent'] > max_threshold]).index,axis= 1)
    dataset = dataset.dropna(axis=0, subset=(missing[missing['Percent'] < min_threshold]).index)
    return dataset 

master_df  = drop_missing(master_df,missing_data_master_df,2,40)
jtd_df     = drop_missing(jtd_df,missing_data_jtd_df,2,40)

KeyError: ignored

In [32]:
#recheck the missing values
new_missing_data = missing_datas(master_df)
print("missing data for 'master_df'")
print(new_missing_data[new_missing_data['Percent']>0.00])

new_missing_data = missing_datas(jtd_df)
print("missing data for 'jtd_df'")
print(new_missing_data[new_missing_data['Percent']>0.00])

missing data for 'master_df'
                     Total    Percent
Sales organization  271769  27.722026
Title               144020  14.690881
Fuel Type            18288   1.865483
missing data for 'jtd_df'
Empty DataFrame
Columns: [Total, Percent]
Index: []


In [0]:
#fill missing values with mode
def Fill_Missing_Values(dataset, cols):
    for col in cols:
      dataset[col].fillna(dataset[col].dropna().mode()[0], inplace = True)
    return dataset
  

master_df = Fill_Missing_Values(dataset=master_df,cols=['Sales organization','Title','Fuel Type'])

In [35]:
# Function to identify numeric features
def numeric_features(dataset):
    numeric_col = dataset.select_dtypes(include=['number']).columns
    return numeric_col

numeric_columns_master_df = numeric_features(master_df)
print("Numeric Features for master_df:")
print(numeric_columns_master_df)

numeric_columns_jtd_df = numeric_features(jtd_df)
print("Numeric Features for jtd_df:")
print(numeric_columns_jtd_df)

print("===="*30)

# Function to identify categorical features
def categorical_features(dataset):
    categorical_col = dataset.select_dtypes(exclude=['number']).columns
    return categorical_col

categorical_columns_master_df = categorical_features(master_df)
print("Categorical Features for master_df:")
print(categorical_columns_master_df)

categorical_columns_jtd_df = categorical_features(jtd_df)
print("categorical Features for jtd_df:")
print(categorical_columns_jtd_df)

Numeric Features for master_df:
Index(['Invoice No', 'Job Card No', 'KMs Reading', 'Labour Total',
       'Misc Total', 'OSL Total', 'Parts Total', 'Pin code', 'Recovrbl Exp',
       'Total Amt Wtd Tax.', 'Partner Type', 'Title'],
      dtype='object')
Numeric Features for jtd_df:
Index(['DBM Order', 'Order Item', 'Order Quantity', 'Net value'], dtype='object')
Categorical Features for master_df:
Index(['CITY', 'Cust Type', 'Customer No.', 'State', 'Gate Pass Time',
       'Invoice Date', 'Invoice Time', 'JobCard Date', 'JobCard Time', 'Make',
       'Model', 'Order Type', 'Plant', 'Plant Name1', 'Print Status',
       'Regn No', 'User ID', 'Business Partner', 'Data Origin',
       'Sales organization', 'Fuel Type'],
      dtype='object')
categorical Features for jtd_df:
Index(['Material', 'Description', 'Item Category', 'Target quantity UoM'], dtype='object')


In [0]:
def to_datetime(dataset, datetime_cols):
  for datetime in datetime_cols:
    dataset[datetime[0] + '-' + datetime[1]] = pd.to_datetime(dataset[datetime[0]]+ ' ' + dataset[datetime[1]])
  return dataset

master_df = to_datetime(master_df, [['Invoice Date','Invoice Time'],['JobCard Date','JobCard Time']])

def job_completion_time(dataset, start_datetime, end_datetime):
    dataset['Job_completion_time(in days)'] = (dataset[start_datetime] - dataset[end_datetime]) / np.timedelta64(1,'D')
    return dataset
    
master_df = job_completion_time(master_df, 'Invoice Date-Invoice Time', 'JobCard Date-JobCard Time')

In [37]:
master_df.sample(3)

,CITY,Cust Type,Customer No.,State,Gate Pass Time,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,OSL Total,Order Type,Parts Total,Pin code,Plant,Plant Name1,Print Status,Recovrbl Exp,Regn No,Total Amt Wtd Tax.,User ID,Business Partner,Partner Type,Data Origin,Title,Sales organization,Fuel Type,Invoice Date-Invoice Time,JobCard Date-JobCard Time,Job_completion_time(in days)
708991,mayiladuthurai,retail,453022,tamil nadu,00:00:00,2017-09-07,7284400213,14:35:05,858550,2017-08-31,16:37:51,75373,1293.05,nissan,0.0,micra,6510.26,paid service,2850.49,609001,x672,x672 - sri santhi vijay motors,no,0.0,tn05af9456,10653.80,x672sa1,464903,1.0,z001,2.0,mfcd,2,2017-09-07 14:35:05,2017-08-31 16:37:51,6.914745
943504,pune,retail,3448,maharashtra,00:00:00,2013-03-31,7000027312,14:04:26,25939,2013-03-18,17:08:25,177096,6499.01,mahindra,580.0,scorpio crde,400.00,paid service,37778.92,411056,bc02,pune,no,0.0,mh12em5050,45257.93,bc02sa1,3796,1.0,z005,2.0,mfcd,2,2013-03-31 14:04:26,2013-03-18 17:08:25,12.872234
470670,pinjore,retail,206896,haryana,00:00:00,2017-03-06,7175802001,11:05:20,627817,2017-03-05,11:30:41,92000,793.95,hyundai,0.0,verna,0.00,running repairs,1937.62,134102,x129,x129 - a.v. automobiles,no,0.0,hr51aa0015,2731.57,x129wm1,217476,1.0,z005,2.0,mfcd,2,2017-03-06 11:05:20,2017-03-05 11:30:41,0.982396


In [0]:
# master_df.to_csv('master.csv', index=False)
# files.download('master.csv')